### Install libraries 

In [ ]:
# !pip install argilla langchain outlines tiktoken transformers ipywidgets jupyter

### Install 

In [10]:
import argilla as rg 

import os 
import random

from langchain.prompts import PromptTemplate
from langchain import LLMChain 
from langchain.output_parsers import CommaSeparatedListOutputParser
from outlines import models
from outlines import generate 

In [6]:
rg.init(
    api_url="http://localhost:6900",
    api_key="argilla.apikey"
)

/Users/adebayoakinlalu/.pyenv/versions/3.11.4/envs/llm-env2/lib/python3.11/site-packages/argilla/client/client.py:167: UserWarning: Default user was detected and no workspace configuration was provided, so the default 'argilla' workspace will be used. If you want to setup another workspace, use the `rg.set_workspace` function or provide a different one on `rg.init`
  warnings.warn(
/Users/adebayoakinlalu/.pyenv/versions/3.11.4/envs/llm-env2/lib/python3.11/site-packages/argilla/client/client.py:195: UserWarning: You're connecting to Argilla Server 1.23.0-dev4 using a different client version (1.25.0).
This may lead to potential compatibility issues during your experience.
To ensure a seamless and optimized connection, we highly recommend aligning your client version with the server version.
  warnings.warn(


### Define a FeedbackDataset
 this example, we will create a synthetic dataset for a banking customer care scenario. We assume that customers will write text requests. These requests should then be classified for sentiment and topics. The topics will be a multi-label classification and can be used to route the request to the correct department. The sentiment will be used using a single-label classification to determine if the request needs to be handled with priority.

In [14]:
sentiment = ['positive', 'negative', 'neutral']
topic = ['new_card', 'mortgage', "application", "payments"]

dataset = rg.FeedbackDataset(
    fields = [rg.TextField(name="text")],
    questions = [
        rg.LabelQuestion(
            name="sentiment",
            title = "what is the sentiment of the message?",
            labels=sentiment
        ),
        rg.MultiLabelQuestion(
            name="topics",
            title="Select the topic(s) of the messages?",
            labels=topic,
            visible_labels=4,
        )
    ],
    guidelines=(
        "This dataset contains messages from a banks customer support chatbot. "
        "The goal is to label the sentiment of the message and the topic of the message. "
    )
)

In [8]:
mistral_model = models.transformers("mistralai/Mistral-7B-Instruct-v0.2")

config.json: 100%|██████████| 596/596 [00:00<00:00, 1.17MB/s]
model.safetensors.index.json: 100%|██████████| 25.1k/25.1k [00:00<00:00, 37.7MB/s]
model-00001-of-00003.safetensors: 100%|██████████| 4.94G/4.94G [12:46<00:00, 6.45MB/s]
model-00002-of-00003.safetensors: 100%|██████████| 5.00G/5.00G [13:14<00:00, 6.30MB/s]
model-00003-of-00003.safetensors: 100%|██████████| 4.54G/4.54G [11:50<00:00, 6.39MB/s]
generation_config.json: 100%|██████████| 111/111 [00:00<00:00, 776kB/s]
tokenizer_config.json: 100%|██████████| 1.46k/1.46k [00:00<00:00, 5.51MB/s]
tokenizer.model: 100%|██████████| 493k/493k [00:00<00:00, 1.75MB/s]
tokenizer.json: 100%|██████████| 1.80M/1.80M [00:01<00:00, 1.59MB/s]
special_tokens_map.json: 100%|██████████| 72.0/72.0 [00:00<00:00, 486kB/s]


In [16]:
generator = generate.text(mistral_model)

In [18]:
from langchain_community.llms import Ollama

llm = Ollama(
    model="mistral"
)

In [19]:
template = (
    "Write a customer review for a bank. "
    "Do that for topic of {topic}. "
    "Do that with one a {sentiment} sentiment."
)
prompt = PromptTemplate(template=template, input_variables=["topic", "sentiment"])
llm_chain_review = LLMChain(prompt=prompt, llm=llm)

def generate_langchain_review():
    return llm_chain_review.predict(
        topic=random.choice(topic),
        sentiment=random.choice(sentiment)
    ).strip()

generate_langchain_review()

'I recently applied for a loan at XYZ Bank and had a positive experience overall. The staff was friendly and knowledgeable, and they provided me with all the information I needed to make an informed decision about my loan. However, the process was quite lengthy and there were a few minor delays along the way. Despite this, I am satisfied with the outcome of my application and would recommend XYZ Bank to others who are looking for a reliable and efficient banking experience. Overall, it was a good experience!'

In [20]:
def generate_n_langchain_reviews(n=2):
    reviews = []
    for n in range(n):
        reviews.append(generate_langchain_review())
    return reviews

langchain_reviews = generate_n_langchain_reviews()
langchain_reviews

["Title: Neutral Review of Bank's Mortgage Services\n\nOverall, I had a neutral experience with the bank's mortgage services. The process was straightforward and efficient, with clear communication throughout the entire process. The staff were friendly and helpful, but it took some time for me to receive my final documents. However, I am happy with the interest rate and payment plan that I was able to secure. Overall, I would recommend the bank's mortgage services to others who are looking for a reliable and efficient mortgage experience.",
 '"I recently applied for a new account at XYZ Bank and I was very pleased with the experience. The online application process was easy to navigate, and I appreciated the clear instructions provided throughout. The customer service team was also very helpful and responsive, answering all of my questions promptly. Overall, I had a positive experience and would recommend XYZ Bank to anyone in need of a reliable banking solution."']

In [ ]:
def generate_n_langchain_reviews(n=5):
    reviews = []
    for n in range(n):
        reviews.append(generate_langchain_review())
    return reviews

langchain_reviews = generate_n_langchain_reviews(5)
langchain_reviews